# Introduction #

In these exercises, you'll explore the feature extraction operations on your own. You'll also see how you can open up a convnet to look at the feature maps deep within its layers.

Run the cell below to get started!

In [ ]:
# Setup feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.computer_vision.ex3 import *

# Apply Transformations

Run the following cell to load an image we'll use for the next few exercises.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image

#TODO: let user select own image (from url maybe?)
image_path = './images/car_feature.jpg'
image = np.array(Image.open(image_path)

plt.figure(figsize=(6, 6))
plt.imshow(image, cmap='gray')
plt.axis('off')
plt.show();

### 1) Define Kernel

First define a kernel. You have your choice of what kind of kernel to apply, but here are some ideas:

In [ ]:
# Edge detection
[[-1, -1, -1],
 [-1, 8, -1],
 [-1, -1, -1]]

# Blur
[[0.0625, 0.125, 0.0625],
 [0.125, 0.25, 0.125],
 [0.0625, 0.125, 0.0625]]
 
# Bottom sobel
[[-1, -2, -1],
 [0, 0, 0],
 [1, 2, 1]]
 
# Emboss
[[-2, -1, 0],
 [-1, 1, 1],
 [0, 1, 2]]

# Sharpen
[[0, -1, 0],
 [-1, 5, -1],
 [0, -1, 0]]

The sum of the entries in the matrix should usually be around 1. Below that and the filtered image will be darker. Above that and it will be lighter.

Feel free to experiment!

In [ ]:
# YOUR CODE HERE: Define this just like you would a numpy array.
kernel = tf.constant(
    ____
)

q_1.check()

In [ ]:
kernel = tf.constant([
    [-2, -1, 0],
    [-1, 1, 1],
    [0, 1, 2],
])
q_1.assert_check_passed()

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_1.hint()
#_COMMENT_IF(PROD)_
q_1.solution()

If you'd like a visualization of your kernel like you saw in the tutorial, you can run this cell.

In [ ]:
visiontools.show_kernel(kernel)

Once you're happy with your solution, run the cell below to set up the extraction operations.

In [ ]:
#TODO: avoid problems with multiple runs
from tensorflow.keras import Sequential
import tensorflow.keras.layers as layers

# Create Layers
model = Sequential([
    layers.Conv2D(filters=1,
                  kernel_size=kernel.shape,
                  padding='same',
                  use_bias=False,
                  input_shape=image.shape),
    layers.Activation('relu'),
    layers.MaxPool2D(pool_size=2,
                     padding='same'),
])
conv2d, relu, maxpool2d = model.layers

# Reformat
image = tf.image.convert_image_dtype(image, dtype=tf.float32)
image = tf.expand_dims(image, axis=0)
kernel = tf.reshape(kernel, [*kernel.shape, 1, 1])

### 2) Apply Convolution

Now apply the filtering operation to your image.

In [ ]:
# YOUR CODE HERE: Set layer weights
____

# YOUR CODE HERE: Apply convolution to image
image_filter = ____

q_2.check()

In [ ]:
q_2.assert_check_passed()

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_2.hint()
#_COMMENT_IF(PROD)_
q_2.solution()

Run the next cell to see the result of the convolution!

In [ ]:
plt.imshow(
    # Reformat for plotting
    tf.squeeze(image_filter)
)
plt.axis('off')
plt.show();

### 3) Apply ReLU

In [ ]:
# YOUR CODE HERE
image_detect = ___

q_3.check()

In [ ]:
q_3.assert_check_passed()

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_3.hint()
#_COMMENT_IF(PROD)_
q_3.solution()

Run the next cell to see the effect of the ReLU function!

In [ ]:
plt.imshow(
    # Reformat for plotting
    tf.squeeze(image_detect)
)
plt.axis('off')
plt.show();

### 4) Apply Pooling

In [ ]:
# YOUR CODE HERE
image_condense = ____

q_4.check()

In [ ]:
q_4.assert_check_passed()

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_4.hint()
#_COMMENT_IF(PROD)_
q_4.solution()

Run the next cell to see what maximum pooling did to the feature!

In [ ]:
plt.imshow(
    # Reformat for plotting
    tf.squeeze(image_detect)
)
plt.axis('off')
plt.show();

### 5) Explore Filters

Use the `show_extraction` function from the `visiontools` module to explore the effect of other kernels. Try one of the kernels at the start of the lesson or invent your own!

In [ ]:
image = np.array(Image.open('./images/car_feature.png'))

kernel = tf.constant([
    # YOUR CODE HERE
    ____
])
q_5.check()

In [ ]:
q_5.assert_check_passed()

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_5.hint()
#_COMMENT_IF(PROD)_
q_5.solution()

Once you're happy with you're kernel, run the cell below to see the feature it extracted!

In [ ]:
visiontools.show_extraction(image, kernel)

You're not limited to the 3x3 kernels we've seen so far. You could try kernels with other dimensions, like 2x2, 6x6, or even 3x5!

### 6) Explore Feature Maps

Use the `show_feature_maps` function from the `visiontools` module to see the feature maps a model produces. Let's start with VGG16. Run the cell below to make a list of the layers you can visualize.

In [ ]:
model = tf.keras.applications.VGG16(include_top=False, input_shape=image.shape)

layer_names = [layer.name for layer in model.layers
               if layer.__class__.__name__ is 'Conv2D']
print(layer_names)

Now choose a layer from the first block.

In [ ]:
# YOUR CODE HERE
layer_b1 = ____
q_6.a.check()

In [ ]:
q_6.a.assert_check_passed()

In [ ]:
q_6.a.solution()

And once you've gotten a solution, run the code below to produce the visualization.

In [ ]:
visiontools.show_feature_maps(image, layer_name=layer_b1)

Now do the same for a layer from the fourth block.

In [ ]:
# YOUR CODE HERE
layer_b4 = ____
q_6.b.check()

In [ ]:
q_6.b.assert_check_passed()

In [ ]:
q_6.b.solution()

And once you've gotten a solution, run the code below to produce the visualization.

In [ ]:
visiontools.show_feature_maps(image, layer_name=layer_b4)

Thought question.

In [ ]:
q_6.c.solution()

# Conclusion #

Here is a fun website where you can interactively explore the effect of image kernels: [Image Kernels Explained Visually](https://setosa.io/ev/image-kernels/).

In the next lesson, we'll go deeper into the details of the convolution and pooling algorithms. We'll see that these two operations have big implications for advanced techniques like **fine tuning** and **data augmentation** -- which you'll learn about soon!